In [1]:
import time
import numpy as np
import gym
import arlenvs

## Environment Creation¶

- uncomment the desired environment
- init env class



In [2]:
# env_name = "MexicanWorld-v0"
# env_name = "ContinuousMexicanWorld-v0"
# env_name = "MarsLander-v0"  # gym.make accepts level=1, level=2 or level=3 as kwarg, to control the difficultly
# env_name = "GettingOverIt-v0"
# env_name = "PlanetWorld-v0"
# env_name = 'GameOfDrones-v0'
# env_name = "FlappyBird-v0"
# env_name = "DronePathFinding-v0"
# env_name = "DronePathTracking-v0"
env_name = "HaxBall-v0"
# unwrapped to get rid of this TimeLimitWrapper, which might reset the environment twice and thus breaks ergodicity
env = gym.make(env_name).unwrapped

/home/sayanta/anaconda3/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


### Random agent for environment testing

In [7]:

state = env.reset()
print("Starting state: ",state)

counter = 0
time_start = time.time()

#while True:
for i in range(100):
  env.render(mode="human")

  action = env.action_space.sample()

  state, reward, done, _ = env.step(action)

  print(reward)

  # For very fast running environments ...
  time.sleep(0.02)

  if done:
    print("Resetting")
    env.reset()
  #end
#end

env.close()

print("time consumed", time.time() - time_start)

Starting state:  [2.5       1.5       0.        0.        1.5       1.5       0.
 0.        4.0999999 1.5      ]
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
-0.001
time consumed 2.5126681327819824


## Init qtable and policy table

- main idea: discretize simple performing a rounding operation

In [6]:


action_size = env.action_space.n
print("Action size ", action_size)


a_sample = env.action_space.sample()
print("action sample: ", np.round(a_sample), a_sample, "\n \n")


state_size = env.observation_space.shape
print("State size ", state_size)
print("State bounds: ", env.observation_space.low, env.observation_space.high)
state_dim = env.observation_space.high -env.observation_space.low +1
print("State dim: ", state_dim )

sample = env.observation_space.sample()
print("state sample: ", np.round(sample), sample)
state_dim_real = np.array([20, 20 , 2])

Action size  9
action sample:  8 8 
 

State size  (4,)
State bounds:  [0. 0. 0. 0.] [0. 0. 0. 0.]
State dim:  [1. 1. 1. 1.]
state sample:  [0. 0. 0. 0.] [0. 0. 0. 0.]


### Monte Carlo

In [5]:
from collections import defaultdict
from time import perf_counter
import sys
import matplotlib.pyplot as plt

In [6]:
env.observation_space

Box(4,)

In [4]:
env.reset()

array([2.5      , 1.5      , 0.       , 0.       , 1.5      , 1.5      ,
       0.       , 0.       , 4.0999999, 1.5      ])

In [7]:
env.step(1)

(array([2.5       , 1.5       , 0.        , 0.        , 1.50375187,
        1.50375187, 0.22511308, 0.22511308, 4.0999999 , 1.5       ]),
 -0.001,
 False,
 {'blue_goal_cnt': 0, 'red_goal_cnt': 0, 'ball_hit_cnt': 0})

In [8]:
train_episodes = 500000
nA = env.action_space.n
Q = defaultdict(lambda: np.zeros(nA, dtype=float))
discount_factor = 0.9  
epsilon = 0.1
env_max_steps = 10000
render = False
test_episodes = 1000
train_rewards = []

In [9]:
def make_epsilon_greedy_policy():
    """
    Creates an epsilon-greedy policy based on a given Q-function and epsilon.
    Returns:
        A function that takes the observation (state) as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.
    """

    def policy_fn(observation):
        prob = np.ones(nA, dtype=float) * epsilon / nA
        optimal_action = np.argmax(Q[tuple(observation)])
        prob[optimal_action] += (1.0 - epsilon)
        return prob

    return policy_fn

In [10]:
def mc_control_train():
    """
    Monte Carlo Control using Epsilon-Greedy policies.
    Finds an optimal epsilon-greedy policy.
    Returns:
        A tuple (Q, policy).
        Q is a dictionary mapping state -> action values.
        policy is a function that takes an observation as an argument and returns
        action probabilities
    """

    # Keeps track of sum and count of returns for each state
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)

    # A nested dictionary that maps state -> (action -> action-value).
    policy_fn = make_epsilon_greedy_policy()
    counter = perf_counter()

    for i_episode in range(1, train_episodes + 1):
        episode = []
        state = env.reset()
        
        if i_episode % 5000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, train_episodes), end="")
            print('no of states explored:', len(Q.keys()))
            print('Time taken:', (perf_counter() - counter) / 60, 'minutes')
            counter = perf_counter()
#             render = True
            sys.stdout.flush()
        else:
            render = False

        reward_episode = 0.0
        for step in range(env_max_steps):
            if state in Q:
                prob_values = policy_fn(state)
                action = np.random.choice(np.arange(nA), p=prob_values)
            else:
                action = env.action_space.sample()

            next_state, reward, done, _ = env.step(action)
            episode.append((next_state, action, reward))
            reward_episode += reward

            if render:
                env.render(mode='rgb_array')

            if done:
                env.close()
                break
                
            state = next_state

        train_rewards.append(reward_episode)
        sa_in_episode = set([(x[0], x[1]) for x in episode])
        for state, action in sa_in_episode:
            sa_pair = (state, action)
            # Find the first occurrence of the (state, action) pair in the episode
            first_occurrence_idx = next(i for i, x in enumerate(episode)
                                        if x[0] == state and x[1] == action)
            # Sum up all rewards since the first occurrence
            G = sum([x[2] * (discount_factor ** i) for i, x in enumerate(episode[first_occurrence_idx:])])
            # Calculate average return for this state over all sampled episodes
            returns_sum[sa_pair] += G
            returns_count[sa_pair] += 1.0
            Q[state][action] = returns_sum[sa_pair] / returns_count[sa_pair]

    return train_rewards

In [11]:
rewards = mc_control_train()

TypeError: unhashable type: 'numpy.ndarray'